In [1]:
# ================== Imports ==================
import os
import random
import glob
import pickle
import datetime
import platform
from pathlib import Path
from time import time
import importlib

import numpy as np
import pandas as pd
import wandb
from tqdm import tqdm
from matplotlib import rc
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation
rc('animation', embed_limit=500)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
from torch.cuda.amp import autocast, GradScaler

from monai.utils import set_determinism
from monai.losses import DiceLoss
from monai.networks.nets import (
    UNet,
    SwinUNETR,
    SegResNet,
    DynUNet,
    AttentionUnet,
    BasicUNet,
    VNet,
    SegResNetDS,
    SegResNetVAE,
    HighResNet,
    BasicUNetPlusPlus
)

# Local imports
def import_module(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

home_dir = '/vast/palmer/home.grace/am3833/fly/segmentation'
utils = import_module('utils', f'{home_dir}/util_files/utils.py')
config = import_module('config', f'{home_dir}/util_files/config.py')
voxel_utils = import_module('voxel_utils', f'{home_dir}/util_files/voxel_utils.py')
segmentation = import_module('segmentation', f'{home_dir}/util_files/segmentation.py')
analysis = import_module('analysis', f'{home_dir}/util_files/analysis.py')
train_helper = import_module('train_helper', f'{home_dir}/util_files/train_helper.py')
dataset = import_module('dataset', f'{home_dir}/dataset.py')

MicrotubuleDataset = dataset.MicrotubuleDataset
hyperparameters = train_helper.hyperparameters

2025-01-13 14:56:55.727046: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-13 14:56:55.877562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-13 14:56:55.972888: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-13 14:56:55.997945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 14:56:56.128271: I tensorflow/core/platform/cpu_feature_guar